# Import Required libraries

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# tensorflow packages
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (Dense, 
                                    Input, 
                                    LSTM,
                                    Dropout)
from tensorflow.keras.optimizers import (Adam, 
                                         AdamW)
from tensorflow.keras.callbacks import (EarlyStopping, 
                                        ModelCheckpoint)
from tensorflow.keras.losses import (SparseCategoricalCrossentropy,
                                     CategoricalCrossentropy)                                         

/Users/kavisanthoshkumar/Library/CloudStorage/OneDrive-IllinoisInstituteofTechnology/Tensorflow/tensorflow_env/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


# Load Train, Validation and Test Dataset

In [2]:
data = np.load("../data/processed/processed_array.npz")

X_train, y_train = data["x_train"], data["y_train"]
X_test, y_test = data["x_test"], data["y_test"]
X_val, y_val = data['x_val'], data["y_val"]

print(f"Shape of X_train : {X_train.shape} and Y_train : {y_train.shape}")
print(f"Shape of X_val : {X_val.shape} and Y_train : {y_val.shape}")
print(f"Shape of X_test : {X_test.shape} and Y_test : {y_test.shape}")

Shape of X_train : (5881, 128, 9) and Y_train : (5881, 6)
Shape of X_val : (1471, 128, 9) and Y_train : (1471, 6)
Shape of X_test : (2947, 128, 9) and Y_test : (2947, 6)


# Create tf.Dataset

In [3]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(64)
validation_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(64)

2025-06-26 21:19:58.114760: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M3 Pro
2025-06-26 21:19:58.114801: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 18.00 GB
2025-06-26 21:19:58.114807: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 6.00 GB
I0000 00:00:1750990798.114817 10228996 pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
I0000 00:00:1750990798.114833 10228996 pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


# Baseline Model

#### Step -1 : Model Architecture

In [4]:
class LSTM_BaselineModel(Model):

    def __init__(self, num_class):
        super(LSTM_BaselineModel, self).__init__()
        
        # LSTM layer
        self.lstm_layer_1 = LSTM(units = 100,
                                 activation = "tanh",
                                 recurrent_activation = 'sigmoid',
                                 input_shape = (),
                                 name = "LSTM")

        # Dense layers
        self.dense_layer_1 = Dense(units = 256, activation = 'leaky_relu')
        self.dense_layer_2 = Dense(units = 128, activation = 'leaky_relu')

        # output layer
        self.output_layer = Dense(units = num_class, activation = "softmax")

    def call(self, inputs):
        # LSTM layer
        x = self.lstm_layer_1(inputs)

        # Dense layer        
        x = self.dense_layer_1(x)
        x = self.dense_layer_2(x)

        # Output layer
        self.out = self.output_layer(x)

        return self.out

model_baseline_lstm = LSTM_BaselineModel(num_class= 6)

/Users/kavisanthoshkumar/Library/CloudStorage/OneDrive-IllinoisInstituteofTechnology/Tensorflow/tensorflow_env/lib/python3.9/site-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


#### Step -2 : Compile the Model

In [5]:
model_baseline_lstm.compile(optimizer = 'adam',
                            loss = CategoricalCrossentropy(),
                            metrics = ["accuracy", "f1_score"])

#### Step-3 Callbacks

In [6]:
# Early Stopping
early_stopping_callback = EarlyStopping(
    monitor = "val_loss",
    patience = 5,
    verbose = 1, 
    restore_best_weights = True, 
    mode = 'min'
)

# Model Check Pointing
model_check_point = ModelCheckpoint(
    filepath = "../model/baseline_model.keras",
    monitor = "val_loss",
    verbose = 1,
    save_best_only = True, 
)

callbacks = [early_stopping_callback, model_check_point]

#### Step 4 : Fit the model

In [8]:
history = model_baseline_lstm.fit(train_dataset, 
                                  epochs = 50, 
                                  batch_size = 32,
                                  validation_data = validation_dataset,
                                  callbacks = callbacks,
                                  verbose = 1)

Epoch 1/50


2025-06-26 21:19:58.615958: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.4938 - f1_score: 0.4453 - loss: 1.2645
Epoch 1: val_loss improved from inf to 0.64640, saving model to ../model/baseline_model.keras
92/92 ━━━━━━━━━━━━━━━━━━━━ 4s 30ms/step - accuracy: 0.4947 - f1_score: 0.4464 - loss: 1.2617 - val_accuracy: 0.7260 - val_f1_score: 0.6946 - val_loss: 0.6464
Epoch 2/50
90/92 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step - accuracy: 0.7390 - f1_score: 0.7196 - loss: 0.6029
Epoch 2: val_loss improved from 0.64640 to 0.43925, saving model to ../model/baseline_model.keras
92/92 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step - accuracy: 0.7403 - f1_score: 0.7210 - loss: 0.6006 - val_accuracy: 0.8178 - val_f1_score: 0.8043 - val_loss: 0.4392
Epoch 3/50
91/92 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - accuracy: 0.8625 - f1_score: 0.8579 - loss: 0.3639
Epoch 3: val_loss improved from 0.43925 to 0.27107, saving model to ../model/baseline_model.keras
92/92 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - accuracy: 0.8628 - f1_score: 0.8583 - loss: 0.3632 

In [ ]:
#### Step